In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import pickle
import time
import tensorflow as tf
import shutil
import numpy as np
from math import ceil
from src import constants as const
import utils.timer
import helpers

dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
columns_num, = np.where(data_train.dtypes != "object")

timer_path = os.path.join(const.dir.timers(), "timer_tgan.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer
    
from tgan.model import TGANModel

dir_data_gen_tgan = os.path.join(const.dir.data_gen(), "TGAN")
model_dir = os.path.join(const.dir.models(), "tgan")

batch_size = 500
retrain=True

In [3]:
print(tf.__version__)
tf.test.is_gpu_available()

1.15.5


True

In [ ]:
from tqdm.auto import tqdm
from IPython.display import clear_output
n_synthetic_datasets = 10
n_synthetic_datasets_existing=0

dataset_dir_tgan = os.path.join(const.dir.data_gen(), "TGAN2") 
model_temp_dir = os.path.join(const.dir.models(), "temp_model")
if os.path.exists(model_temp_dir):
    shutil.rmtree(model_temp_dir)
os.makedirs(dataset_dir_tgan, exist_ok = True)
for i in tqdm(range(n_synthetic_datasets_existing, n_synthetic_datasets), desc = "Generated datasets", disable = False):
    tf.reset_default_graph()
    model_temp_dir_i = os.path.join(model_temp_dir, f"TGAN{i}")
    tgan = TGANModel(columns_num, output = model_temp_dir_i,
                     gpu="0",
                     max_epoch = 100, steps_per_epoch = data_train.shape[0] / batch_size, batch_size=batch_size,
                     num_dis_hidden = 256, #num_dis_layers=1, z_dim=128,
                    restore_session = False, save_checkpoints = True)
    tgan.fit(data_train.reset_index(drop=True))
    fake_train = tgan.sample(ceil(data_train.shape[0] / batch_size) * batch_size).head(data_train.shape[0])
    fake_train.to_csv(os.path.join(dataset_dir_tgan, f"gen{i}.csv"))
    del tgan
    clear_output(wait=True)

[0316 15:31:55 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[0316 15:32:01 @logger.py:90] Argv: C:\Users\Arne\anaconda3\envs\tfv1_gpu\lib\site-packages\ipykernel_launcher.py -f C:\Users\Arne\AppData\Roaming\jupyter\runtime\kernel-621042e5-cafe-43ef-b7bc-e5c120d46132.json
[0316 15:32:01 @model_utils.py:67] List of Trainable Variables: 
name                              shape         #elements
--------------------------------  ----------  -----------
gen/LSTM/go:0                     [1, 100]            100
gen/LSTM/lstm_cell/kernel:0       [500, 400]       200000
gen/LSTM/lstm_cell/bias:0         [400]               400
gen/LSTM/00/FC/W:0                [100, 100]        10000
gen/LSTM/00/FC/b:0                [100]               100
gen/LSTM/00/FC2/W:0               [100, 1]            100
gen/LSTM/00/FC2/b:0               [1]                   1
gen/LSTM/00/attw:0                [1, 1, 1]             1
gen/LSTM/01/FC/W:0                [1

[0316 15:32:01 @base.py:209] Setup callbacks graph ...
[0316 15:32:01 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0316 15:32:01 @summary.py:93] Summarizing collection 'summaries' of size 9.
[0316 15:32:01 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.
[0316 15:32:05 @base.py:230] Creating the session ...
[0316 15:32:07 @base.py:236] Initializing the session ...
[0316 15:32:07 @base.py:243] Graph Finalized.
[0316 15:32:08 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...
[0316 15:32:09 @base.py:275] Start Epoch 1 ...



100%|#####################################################################################|60/60[00:10<00:00, 5.54it/s]

[0316 15:32:20 @base.py:285] Epoch 1 (global_step 60) finished, time:10.8 seconds.


[0316 15:32:23 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN7\model\model-60.
[0316 15:32:23 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.732
[0316 15:32:23 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.3
[0316 15:32:23 @monitor.py:467] GAN_loss/discrim/loss: 0.68543
[0316 15:32:23 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0616
[0316 15:32:23 @monitor.py:467] GAN_loss/gen/klloss: 0.040016
[0316 15:32:23 @monitor.py:467] GAN_loss/gen/loss: 1.0216
[0316 15:32:23 @monitor.py:467] QueueInput/queue_size: 50
[0316 15:32:23 @base.py:275] Start Epoch 2 ...


...

[0316 15:39:12 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN7\model\model-3540.
[0316 15:39:12 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.774
[0316 15:39:12 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.696
[0316 15:39:12 @monitor.py:467] GAN_loss/discrim/loss: 0.54056
[0316 15:39:12 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.3559
[0316 15:39:12 @monitor.py:467] GAN_loss/gen/klloss: 0.081
[0316 15:39:12 @monitor.py:467] GAN_loss/gen/loss: 1.2749
[0316 15:39:12 @monitor.py:467] QueueInput/queue_size: 50
[0316 15:39:12 @base.py:275] Start Epoch 60 ...



100%|#####################################################################################|60/60[00:05<00:00,11.19it/s]

[0316 15:39:18 @base.py:285] Epoch 60 (global_step 3600) finished, time:5.36 seconds.


[0316 15:39:19 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN7\model\model-3600.
[0316 15:39:19 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.876
[0316 15:39:19 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.434
[0316 15:39:19 @monitor.py:467] GAN_loss/discrim/loss: 0.58545
[0316 15:39:19 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.4521
[0316 15:39:19 @monitor.py:467] GAN_loss/gen/klloss: 0.050054
[0316 15:39:19 @monitor.py:467] GAN_loss/gen/loss: 1.4021
[0316 15:39:19 @monitor.py:467] QueueInput/queue_size: 50
[0316 15:39:19 @base.py:275] Start Epoch 61 ...



 30%|#########################5                                                           |18/60[00:01<00:03,11.49it/s]

In [ ]:
?TGANModel

In [ ]:
dat = pd.read_csv(os.path.join(dataset_dir_tgan, "gen8.csv"))
print(dat.shape)
dat